<a href="https://colab.research.google.com/github/RohitKr-codes/Data_Analytics_Sql-Queries/blob/main/Rohit_PlatinumRx.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# run in Google Colab
import sqlite3, pandas as pd
conn = sqlite3.connect('/content/platinumrx.db')
c = conn.cursor()

# ---- SCHEMA: HOTEL (create only once) ----
c.executescript("""
DROP TABLE IF EXISTS users;
DROP TABLE IF EXISTS bookings;
DROP TABLE IF EXISTS booking_commercials;
DROP TABLE IF EXISTS items;

CREATE TABLE users (
  user_id TEXT PRIMARY KEY,
  name TEXT,
  phone_number TEXT,
  mail_id TEXT,
  billing_address TEXT
);

CREATE TABLE bookings (
  booking_id TEXT PRIMARY KEY,
  booking_date TEXT,  -- use ISO datetime strings
  room_no TEXT,
  user_id TEXT
);

CREATE TABLE booking_commercials (
  id TEXT PRIMARY KEY,
  booking_id TEXT,
  bill_id TEXT,
  bill_date TEXT,
  item_id TEXT,
  item_quantity REAL
);

CREATE TABLE items (
  item_id TEXT PRIMARY KEY,
  item_name TEXT,
  item_rate REAL
);
""")

# ---- SAMPLE DATA (replace with assignment sample rows) ----
c.executemany("INSERT OR IGNORE INTO users VALUES (?,?,?,?,?)", [
  ('21wrcxuy-67erfn','John Doe','97XXXXXXXX','john.doe@example.com','XX, Street Y, ABC City')
])
c.executemany("INSERT OR IGNORE INTO bookings VALUES (?,?,?,?)", [
  ('bk-09f3e-95hj','2021-09-23 07:36:48','rm-bhf9-aerjn','21wrcxuy-67erfn'),
  ('bk-nov-01','2021-11-05 10:10:00','rm-101','21wrcxuy-67erfn')
])
c.executemany("INSERT OR IGNORE INTO items VALUES (?,?,?)", [
  ('itm-a9e8-q8fu','Tawa Paratha',18),
  ('itm-a07vh-aer8','Mix Veg',89)
])
c.executemany("INSERT OR IGNORE INTO booking_commercials VALUES (?,?,?,?,?,?)", [
  ('q34r-3q4o8-q34u','bk-09f3e-95hj','bl-0a87y-q340','2021-09-23 12:03:22','itm-a9e8-q8fu',3),
  ('qq-nov','bk-nov-01','bl-nov-1','2021-11-05 12:00:00','itm-a07vh-aer8',2)
])
conn.commit()

# ---- HELPER: run query & show ----
def run(q):
    df = pd.read_sql_query(q, conn)
    display(df)

# Now you can call run("SELECT ...") with the queries provided below
print("DB ready. Use run(SQL) to execute queries.")


DB ready. Use run(SQL) to execute queries.


## **HOTEL: SQL solutions**

1. For every user → user_id and last booked room_no

In [ ]:
run("""
SELECT b.user_id, b.room_no, b.booking_date
FROM bookings b
JOIN (
  SELECT user_id, MAX(booking_date) AS last_booking
  FROM bookings
  GROUP BY user_id
) lb ON b.user_id = lb.user_id AND b.booking_date = lb.last_booking;
""")


NameError: name 'run' is not defined

2. booking_id and total billing amount of every booking created in November 2021

In [ ]:
run(""" SELECT bc.booking_id,
       SUM(bc.item_quantity * it.item_rate) AS total_amount
FROM booking_commercials bc
JOIN items it ON bc.item_id = it.item_id
JOIN bookings b ON bc.booking_id = b.booking_id
WHERE date(b.booking_date) BETWEEN '2021-11-01' AND '2021-11-30'
GROUP BY bc.booking_id;
""")


3. bill_id and bill amount of all bills raised in October 2021 having bill amount > 1000

In [ ]:
run("""SELECT bc.bill_id,
       SUM(bc.item_quantity * it.item_rate) AS bill_amount
FROM booking_commercials bc
JOIN items it ON bc.item_id = it.item_id
WHERE date(bc.bill_date) BETWEEN '2021-10-01' AND '2021-10-31'
GROUP BY bc.bill_id
HAVING bill_amount > 1000;
""")

4. Most-ordered and least-ordered item of each month of 2021
(We compute monthly item totals, then rank)

In [ ]:
run("""WITH monthly AS (
  SELECT strftime('%Y-%m', bc.bill_date) AS month,
         bc.item_id,
         SUM(bc.item_quantity) AS qty
  FROM booking_commercials bc
  WHERE strftime('%Y', bc.bill_date) = '2021'
  GROUP BY month, bc.item_id
),
ranked_max AS (
  SELECT month, item_id, qty,
         ROW_NUMBER() OVER (PARTITION BY month ORDER BY qty DESC) AS rn_desc
  FROM monthly
),
ranked_min AS (
  SELECT month, item_id, qty,
         ROW_NUMBER() OVER (PARTITION BY month ORDER BY qty ASC) AS rn_asc
  FROM monthly
)
SELECT 'most' AS type, rm.month, rm.item_id, i.item_name, rm.qty
FROM ranked_max rm JOIN items i ON rm.item_id = i.item_id
WHERE rm.rn_desc = 1
UNION ALL
SELECT 'least' AS type, rn.month, rn.item_id, i.item_name, rn.qty
FROM ranked_min rn JOIN items i ON rn.item_id = i.item_id
WHERE rn.rn_asc = 1
ORDER BY month, type;
""")

5. Customers with the 2nd highest bill value of each month of 2021
(We calculate bill totals per customer per month, then pick rank=2)

In [ ]:
run("""WITH bill_per_customer AS (
  SELECT b.user_id,
         strftime('%Y-%m', bc.bill_date) AS month,
         SUM(bc.item_quantity * it.item_rate) AS bill_total
  FROM booking_commercials bc
  JOIN bookings b ON bc.booking_id = b.booking_id
  JOIN items it ON bc.item_id = it.item_id
  WHERE strftime('%Y', bc.bill_date) = '2021'
  GROUP BY b.user_id, month
),
ranked AS (
  SELECT *,
         DENSE_RANK() OVER (PARTITION BY month ORDER BY bill_total DESC) AS dr
  FROM bill_per_customer
)
SELECT month, user_id, bill_total
FROM ranked
WHERE dr = 2;
""")

NameError: name 'run' is not defined

# **CLINIC: SQL solutions**

In [ ]:
# Run this entire cell in Google Colab (after you already ran the DB setup earlier)
import sqlite3, pandas as pd
conn = sqlite3.connect('/content/platinumrx.db')
c = conn.cursor()

# helper function (re-define if needed)
def run(q):
    df = pd.read_sql_query(q, conn)
    display(df)

# 1) show existing tables (quick check)
print("Existing tables:")
for row in c.execute("SELECT name FROM sqlite_master WHERE type='table';").fetchall():
    print("-", row[0])
print("\n---\n")

# 2) create clinic schema if not exists
c.executescript("""
CREATE TABLE IF NOT EXISTS clinics (
  cid TEXT PRIMARY KEY,
  clinic_name TEXT,
  city TEXT,
  state TEXT,
  country TEXT
);

CREATE TABLE IF NOT EXISTS customer (
  uid TEXT PRIMARY KEY,
  name TEXT,
  mobile TEXT
);

CREATE TABLE IF NOT EXISTS clinic_sales (
  oid TEXT PRIMARY KEY,
  uid TEXT,
  cid TEXT,
  amount REAL,
  datetime TEXT,
  sales_channel TEXT
);

CREATE TABLE IF NOT EXISTS expenses (
  eid TEXT PRIMARY KEY,
  cid TEXT,
  description TEXT,
  amount REAL,
  datetime TEXT
);
""")
conn.commit()

# 3) insert sample data (only if table is empty) - replace with your assignment data as needed
# We'll guard inserts with simple existence checks so re-running won't duplicate
if c.execute("SELECT COUNT(*) FROM clinic_sales").fetchone()[0] == 0:
    c.executemany("INSERT INTO clinics VALUES (?,?,?,?,?)", [
        ('cnc-0100001','XYZ clinic','Mumbai','Maharashtra','India'),
        ('cnc-0100002','ABC clinic','Pune','Maharashtra','India'),
        ('cnc-0100003','QWE clinic','Delhi','Delhi','India'),
    ])
    c.executemany("INSERT INTO customer VALUES (?,?,?)", [
        ('bk-09f3e-95hj','Jon Doe','97XXXXXXXX'),
        ('bk-09f3e-95hk','Jane Roe','98YYYYYYYY'),
    ])
    c.executemany("INSERT INTO clinic_sales VALUES (?,?,?,?,?,?)", [
        ('ord-00100-00100','bk-09f3e-95hj','cnc-0100001',24999,'2021-09-23 12:03:22','sodat'),
        ('ord-00100-00101','bk-09f3e-95hk','cnc-0100002',15000,'2021-06-10 09:12:00','walkin'),
        ('ord-00100-00102','bk-09f3e-95hj','cnc-0100001',5000,'2021-11-05 13:00:00','online'),
        ('ord-00100-00103','bk-09f3e-95hk','cnc-0100003',9000,'2020-12-01 10:00:00','walkin'),
    ])
    c.executemany("INSERT INTO expenses VALUES (?,?,?,?,?)", [
        ('exp-0100-00100','cnc-0100001','first-aid supplies',557,'2021-09-23 07:36:48'),
        ('exp-0100-00101','cnc-0100002','rent',2000,'2021-06-01 00:00:00'),
    ])
    conn.commit()
    print("Inserted sample clinic data (you can replace these with assignment rows).")
else:
    print("clinic_sales already has data; skipping sample inserts.")

print("\n---\nNow running the revenue-by-sales-channel query for 2021:\n")




Existing tables:
- users
- bookings
- booking_commercials
- items
- clinics
- customer
- clinic_sales
- expenses

---

clinic_sales already has data; skipping sample inserts.

---
Now running the revenue-by-sales-channel query for 2021:



1. Revenue from each sales channel in a given year (replace '2021')

In [ ]:
run("""
SELECT sales_channel, SUM(amount) AS revenue
FROM clinic_sales
WHERE strftime('%Y', datetime) = '2021'
GROUP BY sales_channel
ORDER BY revenue DESC;
""")


,sales_channel,revenue
0,sodat,24999.0
1,walkin,15000.0
2,online,5000.0


2. Top 10 most valuable customers for a given year

In [ ]:
run("""SELECT uid, SUM(amount) AS total_spend
FROM clinic_sales
WHERE strftime('%Y', datetime) = '2021'
GROUP BY uid
ORDER BY total_spend DESC
LIMIT 10;
""")

,uid,total_spend
0,bk-09f3e-95hj,29999.0
1,bk-09f3e-95hk,15000.0


3. Month-wise revenue, expense, profit, status (profitable / not-profitable) for a given year

In [ ]:

run("""
WITH revenue AS (
  SELECT strftime('%Y-%m', datetime) AS month, SUM(amount) AS revenue
  FROM clinic_sales
  WHERE strftime('%Y', datetime) = '2021'
  GROUP BY month
),
expense AS (
  SELECT strftime('%Y-%m', datetime) AS month, SUM(amount) AS expense
  FROM expenses
  WHERE strftime('%Y', datetime) = '2021'
  GROUP BY month
),
months AS (
  SELECT month FROM revenue
  UNION
  SELECT month FROM expense
)
SELECT m.month,
       COALESCE(r.revenue, 0) AS revenue,
       COALESCE(e.expense, 0) AS expense,
       COALESCE(r.revenue, 0) - COALESCE(e.expense, 0) AS profit,
       CASE WHEN COALESCE(r.revenue, 0) - COALESCE(e.expense, 0) > 0 THEN 'profitable' ELSE 'not-profitable' END AS status
FROM months m
LEFT JOIN revenue r ON m.month = r.month
LEFT JOIN expense e ON m.month = e.month
ORDER BY m.month;
""")


,month,revenue,expense,profit,status
0,2021-06,15000.0,2000.0,13000.0,profitable
1,2021-09,24999.0,557.0,24442.0,profitable
2,2021-11,5000.0,0.0,5000.0,profitable


In [ ]:
year = '2021'   # change to any year
run(f"""
WITH revenue AS (
  SELECT strftime('%Y-%m', datetime) AS month, SUM(amount) AS revenue
  FROM clinic_sales
  WHERE strftime('%Y', datetime) = '{year}'
  GROUP BY month
),
expense AS (
  SELECT strftime('%Y-%m', datetime) AS month, SUM(amount) AS expense
  FROM expenses
  WHERE strftime('%Y', datetime) = '{year}'
  GROUP BY month
),
months AS (
  SELECT month FROM revenue
  UNION
  SELECT month FROM expense
)
SELECT m.month,
       COALESCE(r.revenue, 0) AS revenue,
       COALESCE(e.expense, 0) AS expense,
       COALESCE(r.revenue, 0) - COALESCE(e.expense, 0) AS profit,
       CASE WHEN COALESCE(r.revenue, 0) - COALESCE(e.expense, 0) > 0 THEN 'profitable' ELSE 'not-profitable' END AS status
FROM months m
LEFT JOIN revenue r ON m.month = r.month
LEFT JOIN expense e ON m.month = e.month
ORDER BY m.month;
""")


,month,revenue,expense,profit,status
0,2021-06,15000.0,2000.0,13000.0,profitable
1,2021-09,24999.0,557.0,24442.0,profitable
2,2021-11,5000.0,0.0,5000.0,profitable


4. For each city find the most profitable clinic for a given month

In [ ]:

month = '2021-09'

run(f"""
WITH clinic_profit AS (
  -- revenue and expense per clinic for the chosen month
  SELECT cs.cid,
         SUM(cs.amount) AS revenue,
         COALESCE(e.total_expense,0) AS expense,
         SUM(cs.amount) - COALESCE(e.total_expense,0) AS profit
  FROM clinic_sales cs
  LEFT JOIN (
    SELECT cid, strftime('%Y-%m', datetime) AS month, SUM(amount) AS total_expense
    FROM expenses
    GROUP BY cid, month
  ) e
    ON cs.cid = e.cid
    AND strftime('%Y-%m', cs.datetime) = e.month
  WHERE strftime('%Y-%m', cs.datetime) = '{month}'
  GROUP BY cs.cid
),
joined AS (
  -- bring clinic metadata (city) into the same rowset
  SELECT cp.cid, cp.revenue, cp.expense, cp.profit, cl.city
  FROM clinic_profit cp
  JOIN clinics cl ON cp.cid = cl.cid
)
SELECT city, cid, profit
FROM (
  -- rank clinics within each city by profit (highest first)
  SELECT *, ROW_NUMBER() OVER (PARTITION BY city ORDER BY profit DESC) AS rn
  FROM joined
) t
WHERE rn = 1
ORDER BY city;
""")


,city,cid,profit
0,Mumbai,cnc-0100001,24442.0


5. For each state find the second least profitable clinic for a given month

In [ ]:

month = '2021-09'

run(f"""
WITH rv AS (
  SELECT cid, SUM(amount) AS revenue
  FROM clinic_sales
  WHERE strftime('%Y-%m', datetime) = '{month}'
  GROUP BY cid
),
ex AS (
  SELECT cid, SUM(amount) AS expense
  FROM expenses
  WHERE strftime('%Y-%m', datetime) = '{month}'
  GROUP BY cid
),
clinic_profit AS (
  SELECT cl.state,
         rv.cid,
         (rv.revenue - COALESCE(ex.expense, 0)) AS profit
  FROM rv
  LEFT JOIN ex ON rv.cid = ex.cid
  JOIN clinics cl ON rv.cid = cl.cid
)
SELECT state, cid, profit
FROM (
  SELECT state, cid, profit,
         ROW_NUMBER() OVER (PARTITION BY state ORDER BY profit ASC) AS rn
  FROM clinic_profit
) t
WHERE rn = 2
ORDER BY state;
""")


,state,cid,profit


# **Python scripts**

1. 01_Time_Converter.py

In [ ]:
def minutes_to_human(minutes: int) -> str:
    if minutes < 0:
        raise ValueError("minutes must be >= 0")
    hrs = minutes // 60
    mins = minutes % 60
    hr_s = "hr" if hrs==1 else "hrs"
    min_s = "minute" if mins==1 else "minutes"
    if hrs and mins:
        return f"{hrs} {hr_s} {mins} {min_s}"
    if hrs:
        return f"{hrs} {hr_s}"
    return f"{mins} {min_s}"

# examples
print(minutes_to_human(130))  # "2 hrs 10 minutes"
print(minutes_to_human(110))  # "1 hr 50 minutes"


2 hrs 10 minutes
1 hr 50 minutes


2. 02_Remove_Duplicates.py (loop-based, preserves first occurrence order)

In [ ]:
def remove_duplicates(s: str) -> str:
    result = ""
    for ch in s:
        if ch not in result:
            result += ch
    return result

# examples
print(remove_duplicates("aabbccabc"))  # 'abc'
print(remove_duplicates("RohitRohit")) # 'Rohit'


abc
Rohit


# **Spreadsheet Proficiency**

1. Populate 'ticket_created_at'
This task is solved by joining (or merging) the feedbacks table with the ticket table based on the common key, cms_id.

  Logic: Use pd.merge with the cms_id as the key to pull the created_at timestamp from the ticket DataFrame into the feedbacks DataFrame.

2. Fetch outlet-wise count of tickets created AND closed
This task is solved by creating boolean (helper) columns based on date/time comparison and then aggregating the counts using groupby and sum.

a. Same Day: Check if the date part of created_at equals the date part of closed_at.

b. Same Hour of Same Day: Check the "Same Day" condition AND if the hour part of the two timestamps are equal.

Aggregation: Group the results by outlet_id and sum the boolean columns (where True equals 1).

In [ ]:
import pandas as pd
from io import StringIO

# --- 1. Data Setup (Simulating two worksheets) ---

# Sample data for 'ticket' table (using additional data for better counting examples)
ticket_data = """
ticket_id,created_at,closed_at,outlet_id,cms_id
isu-sjd-457,"2021-08-19 16:45:43","2021-08-22 12:33:32",wrqy-juv-978,vew-iuvd-12
qer-fal-092,"2021-08-21 11:09:22","2021-08-21 17:13:45",8woh-k3u-23b,
ticket-003,"2021-08-25 09:00:00","2021-08-25 09:15:00",wrqy-juv-978,cms-xyz-3
ticket-004,"2021-08-26 14:30:00","2021-08-26 15:30:00",8woh-k3u-23b,cms-abc-4
ticket-005,"2021-08-26 14:30:00","2021-08-26 14:59:00",8woh-k3u-23b,cms-def-5
ticket-006,"2021-09-01 10:00:00","2021-09-01 10:45:00",8woh-k3u-23b,cms-fgh-6
"""
ticket_df = pd.read_csv(StringIO(ticket_data))

# Sample data for 'feedbacks' table
feedbacks_data = """
cms_id,feedback_at,feedback_rating,ticket_created_at
vew-iuvd-12,"2021-08-21 13:26:48",3,(to be filled)
cms-xyz-3,"2021-08-27 10:00:00",5,(to be filled)
cms-not-in-ticket,"2021-08-28 11:00:00",4,(to be filled)
"""
feedbacks_df = pd.read_csv(StringIO(feedbacks_data))

# Convert to datetime objects for accurate comparison
ticket_df['created_at'] = pd.to_datetime(ticket_df['created_at'])
ticket_df['closed_at'] = pd.to_datetime(ticket_df['closed_at'])


# --- 1. Populate 'ticket_created_at' in feedbacks table ---

# Create a lookup table with only the necessary columns
lookup_df = ticket_df[['cms_id', 'created_at']].rename(columns={'created_at': 'ticket_created_at'})

# Merge the feedbacks table with the lookup table on 'cms_id'
feedbacks_filled_df = pd.merge(
    feedbacks_df.drop(columns=['ticket_created_at']), # Drop the placeholder
    lookup_df,
    on='cms_id',
    how='left' # Keep all rows from the feedbacks table
)

# --- 2. Fetch outlet-wise count of tickets created AND closed ---

# 2.a. Same Day? (Helper Column)
ticket_df['same_day'] = ticket_df['created_at'].dt.date == ticket_df['closed_at'].dt.date

# 2.b. Same Hour? (Helper Column)
# Check if date is the same AND hour is the same
ticket_df['same_hour'] = (ticket_df['same_day']) & (
    ticket_df['created_at'].dt.hour == ticket_df['closed_at'].dt.hour
)

# 2.c. Aggregation: Group by outlet_id and sum the boolean columns
aggregation_result = ticket_df.groupby('outlet_id').agg(
    tickets_closed_same_day=('same_day', 'sum'),
    tickets_closed_same_hour=('same_hour', 'sum')
).reset_index()